<a href="https://colab.research.google.com/github/kikiru328/enterprise_analysis/blob/main/modified_amt_p_cust_by_prod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

# 한글폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
## 돌리고 런타임 재시작
## 드라이브 마운트, 한글폰트설치코드 제외 셀 실행

In [71]:
#  Directory 변경 (모듈이용)
%pwd
%cd '../content/notebooks'

# module import
## Data Load
from pickle5 import pickle 

## Basic module
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

import warnings
warnings.filterwarnings('ignore')

[Errno 2] No such file or directory: '../content/notebooks'
/content/drive/My Drive/Colab Notebooks


In [3]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/data/EDA/기존_purprd_cust.pkl','rb') as file:
  df = pickle.load(file)


In [4]:
df

,성별,연령대,거주지역,제휴사,멤버십가입개수,영수증번호,고객번호,점포코드,구매일자,구매년도,구매월,구매시간,대분류명,중분류명,소분류명,구매금액
0,F,50대,경기 김포시,A,0,2757986,15148,10,2014-09-20,2014,9,20,가구/인테리어,일용잡화,위생세제,1000
1,F,50대,경기 김포시,A,0,2757986,15148,10,2014-09-20,2014,9,20,가공식품,가공식품,일반가공식품,4100
2,F,60세이상,경기 김포시,A,0,2757987,400,10,2014-09-20,2014,9,12,신선식품,농산물,채소,5000
3,F,60세이상,경기 김포시,A,0,2757987,400,10,2014-09-20,2014,9,12,가공식품,젓갈/반찬,벌크김치,10200
4,F,50대,경기 김포시,A,0,2757988,2043,10,2014-09-20,2014,9,13,가구/인테리어,일용잡화,위생세제,3950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593025,M,30대,경기 김포시,D,1,253839,15835,40,2015-10-26,2015,10,13,가공식품,음료,일반차음료,3000
28593026,M,30대,경기 김포시,D,1,253839,15835,40,2015-10-26,2015,10,13,가공식품,음료,커피음료,1300
28593027,M,50대,울산 동구,D,0,253840,2070,40,2015-10-26,2015,10,16,일상용품,헤어케어,샴푸,59700
28593028,M,30대,경기 김포시,D,1,253841,15835,40,2015-10-26,2015,10,23,가공식품,과자,하드캔디,2000


In [152]:
df_1 = df[(df['구매년도']==2014)&(df['구매월']>=1) & (df['구매월']<=6)] 
df_2= df[(df['구매년도']==2014)&(df['구매월']>=7) & (df['구매월']<=12)]
df_3= df[(df['구매년도']==2015)&(df['구매월']>=1) & (df['구매월']<=6)]
df_4 = df[(df['구매년도']==2015)&(df['구매월']>=7) & (df['구매월']<=12)]

In [208]:
df_1[df_1['고객번호']==1][['영수증번호']].nunique() # True

영수증번호    159
dtype: int64

In [ ]:
# 구매횟수
df = 'df_1'
countpur = df_1[['고객번호','영수증번호','대분류명','구매금액']]
countpur = countpur.groupby(['고객번호','영수증번호','대분류명']).sum()
countpur = countpur.reset_index()
countpur = countpur.groupby(['영수증번호','고객번호']).sum()
countpur = countpur.reset_index()
countpur['거래횟수'] = 1 
countpur = countpur.groupby('고객번호').sum()
countpur = countpur.drop('영수증번호',axis=1)
countpur['객단가'] = countpur['구매금액'] / countpur['거래횟수']
# cols = list(countpur.columns)
# for i in cols:
#   countpur[f'{df}_{i}'] = countpur[i]
# countpur = countpur.drop(cols,axis=1)
countpur

In [215]:
def countpur(df,a):
  countpur = df_1[['고객번호','영수증번호','대분류명','구매금액']]
  countpur = countpur.groupby(['고객번호','영수증번호','대분류명']).sum()
  countpur = countpur.reset_index()
  countpur = countpur.groupby(['영수증번호','고객번호']).sum()
  countpur = countpur.reset_index()
  countpur['거래횟수'] = 1 
  countpur = countpur.groupby('고객번호').sum()
  countpur = countpur.drop('영수증번호',axis=1)
  countpur['객단가'] = countpur['구매금액'] / countpur['거래횟수']
  cols = list(countpur.columns)
  for i in cols:
    countpur[f'{a}_{i}'] = countpur[i]
  countpur = countpur.drop(cols,axis=1)
  return countpur

In [216]:
H1 = countpur(H1,'H1')
H2 = countpur(H2,'H2')
H3 = countpur(H3,'H3')
H4 = countpur(H4,'H4')

H1

,H1_구매금액,H1_거래횟수,H1_객단가
고객번호,,,
1,19925355,159,125316.698113
2,24323230,188,129378.882979
3,851215,128,6650.117188
4,5042106,137,36803.693431
5,5460150,220,24818.863636
...,...,...,...
19356,170160,12,14180.000000
19357,91950,7,13135.714286
19359,43190,5,8638.000000


In [217]:
def to_df(a,b,c):
  test = pd.concat([a,b],axis=1)
  final = pd.concat([test,c],axis=1)
  return final

In [218]:
final = to_df(H1,H2,H3)

In [221]:
ficols = list(final.columns)
ficols = ['H1_구매금액',
          'H2_구매금액',
          'H3_구매금액',
          'H1_거래횟수',
          'H2_거래횟수',
          'H3_거래횟수',
          'H1_객단가',
          'H2_객단가',
          'H3_객단가']

In [222]:
final = final[ficols]
final

,H1_구매금액,H2_구매금액,H3_구매금액,H1_거래횟수,H2_거래횟수,H3_거래횟수,H1_객단가,H2_객단가,H3_객단가
고객번호,,,,,,,,,
1,19925355,19925355,19925355,159,159,159,125316.698113,125316.698113,125316.698113
2,24323230,24323230,24323230,188,188,188,129378.882979,129378.882979,129378.882979
3,851215,851215,851215,128,128,128,6650.117188,6650.117188,6650.117188
4,5042106,5042106,5042106,137,137,137,36803.693431,36803.693431,36803.693431
5,5460150,5460150,5460150,220,220,220,24818.863636,24818.863636,24818.863636
...,...,...,...,...,...,...,...,...,...
19356,170160,170160,170160,12,12,12,14180.000000,14180.000000,14180.000000
19357,91950,91950,91950,7,7,7,13135.714286,13135.714286,13135.714286
19359,43190,43190,43190,5,5,5,8638.000000,8638.000000,8638.000000


In [223]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/진짜진짜최종/123(train)/amt_c_123.pkl','wb') as save:
  pickle.dump(final,save)

In [224]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/진짜진짜최종/4(test)/amt_c_4.pkl','wb') as save:
  pickle.dump(H4,save)